In [23]:
import stanza
# stanza.download('en')

In [24]:
import re
import json

In [25]:
import pandas as pd
import sys
import io

In [26]:
nlp = stanza.Pipeline('en') # initialize English neural pipeline

2022-04-12 14:01:35 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-04-12 14:01:35 INFO: Use device: cpu
2022-04-12 14:01:35 INFO: Loading: tokenize
2022-04-12 14:01:35 INFO: Loading: pos
2022-04-12 14:01:35 INFO: Loading: lemma
2022-04-12 14:01:35 INFO: Loading: depparse
2022-04-12 14:01:36 INFO: Loading: sentiment
2022-04-12 14:01:36 INFO: Loading: constituency
2022-04-12 14:01:37 INFO: Loading: ner
2022-04-12 14:01:37 INFO: Done loading processors!


In [27]:
# Checklist:

# Websites? Find and replace .com or http

# Location Info -> NLP

# DONE:
# Company / Organization Names -> NLP
# Names -> NLP 
# Emails -> Regex 
# Pronouns -> Dictionary replace
# Dates -> Regex
# Phone Numbers -> Regex
# Gender -> Dictionary replace



In [28]:
import PyPDF2

In [29]:

# Code from https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
def read_pdf(filename):
    # creating a pdf file object
    pdfFileObj = open(filename, 'rb')

    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # printing number of pages in pdf file
    print(pdfReader.numPages)

    # creating a page object
    pageObj = pdfReader.getPage(0)

    # extracting text from page
    print(pageObj.extractText())

    #to save the text and run NLP instead of print
    resume_text = pageObj.extractText()

    # closing the pdf file object
    pdfFileObj.close()

#code from - https://stackoverflow.com/questions/55220455/convert-from-pdf-to-text-lines-and-words-are-broken
def extract_with_pdf_miner():
    from io import StringIO
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    import os
    import sys, getopt

    # converts pdf, returns its text content as a string
    def convert(fname, pages=None):
        if not pages:
            pagenums = set()
        else:
            pagenums = set(pages)

        output = io.StringIO()
        manager = PDFResourceManager()
        converter = TextConverter(manager, output, laparams=LAParams())
        interpreter = PDFPageInterpreter(manager, converter)

        infile = open(fname, 'rb')
        for page in PDFPage.get_pages(infile, pagenums):
            interpreter.process_page(page)
        infile.close()
        converter.close()
        text = output.getvalue()
        output.close
        return text

        # converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir

    def convertMultiple(pdfDir, txtDir):
        if pdfDir == "": pdfDir = os.getcwd() + "\\"  # if no pdfDir passed in
        for pdf in os.listdir(pdfDir):  # iterate through pdfs in pdf directory
            fileExtension = pdf.split(".")[-1]
            if fileExtension == "pdf":
                pdfFilename = pdfDir + pdf
                text = convert(pdfFilename)  # get string of text content of pdf
                textFilename = txtDir + pdf + ".txt"
                textFile = open(textFilename, "w")  # make text file
                textFile.write(text)  # write text to text file

    # set paths accordingly:
    pdfDir = "./"
    txtDir = "./"
    convertMultiple(pdfDir, txtDir)

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    pass

In [79]:
def core_nlp(input_paragraph, looking_for=None):
        
    # First, handle pronoun replacement
    replacements = {'he': ' [THEY]',
                      'him': ' [THEM]',
                      'his': ' [THEIR]',
                      'she': ' [THEY]',
                      'her': ' [THEM]',
                      'hers': ' [THEIR]',
                      'man': '[PERSON]',
                      'woman': '[PERSON]',
                      'men': '[PEOPLE]',
                      'women': '[PEOPLE]'
                     }

    def replace_pronouns(match):
        return replacements[match.group(0)]

    regex = '|'.join(r'\b%s\b' % re.escape(s) for s in replacements)
    input_paragraph = re.sub(regex, replace_pronouns, input_paragraph)
    
    
    # Redact Emails
    regex_email = '[a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+'
    lst_emails = re.findall(regex_email, input_paragraph)
    for email in lst_emails:
        input_paragraph = input_paragraph.replace(email,"[EMAIL]")

    # Redact Phone numbers
    regex_phone = '\d{3}[-\.\s]+\d{3}[-\.\s]+\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]+\d{4}'
    lst_phones = re.findall(regex_phone, input_paragraph)
    for phone in lst_phones:
        input_paragraph = input_paragraph.replace(phone,"[PHONE]")

    # Redact Dates
    #regex_date = "((Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4})"
    #regex_date = '[\d]{1,2}[/-][\d]{1,2}[/-][\d]{4}'
    regex_date = "([\d]{1,2}[/-][\d]{1,2}[/-][\d]{4})|((Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+\d{1,2},\s+\d{4})|((Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s?[\d{1,2}]?[,]?\s+\d{4})|(\d{4}[-]+\d{4})"
    lst_dates = re.findall(regex_date, input_paragraph)
    for date in lst_dates:
        date = max(date, key = len)
        input_paragraph = input_paragraph.replace(date,"[DATE]")
    
    # Redact Address/Location, Company/Organization, and Names using stanza
    try:
        doc = nlp(input_paragraph)
        lst_entities = {
            'FAC': '[ADDRESS]',
            'GPE': '[LOCATION]',
            'PERSON': '[PERSON]',
            'ORG': '[ORGANIZATION]',
            'LOC': '[LOCATION]'
        }
        for sentence in doc.sentences:
            for i in sentence.ents:
                if i.type in lst_entities and i.text not in ['PERSON', 'EMAIL']:
                    input_paragraph = input_paragraph.replace(i.text,lst_entities[i.type])
    except:
        print("corenlp didn't work..")
    
    


    print(input_paragraph)


In [81]:
# Testing Phone Numbers:
input_paragraph = "303-888-6666 (303) 888 6666 (303) 888-6666 303 888 6666 303.888.6666"
core_nlp(input_paragraph)

[PHONE] [PHONE] [PHONE] [PHONE] [PHONE]


In [82]:
# Testing Emails:
input_paragraph = """bryce.schumacher8@gmail.com brsc9114@colorado.edu bryce.schumacher@colorado.edu thoughtlessEduardo14@charter.net Meaganfaithful@yahoo.in comfortableCassandra46@bol.com.br cloudyLisa88@yahoo.fr Tristanperfect@ymail.com enchantingKelly@arcor.de grumpyOlivia0@ymail.com Audreysleepy@live.ca goodJulie@hotmail.com anxiousKrystal@gmx.de grotesqueAlisha54@googlemail.com arrogantGerald@yahoo.com.br cleanAndres@live.it mistyTracy@home.nl annoyingDominique7@yahoo.es energeticAndrew@hetnet.nl boredGrant11@bluewin.ch unsightlyJason45@live.com.au oddLinda33@wanadoo.fr gorgeousPatricia70@wanadoo.fr gentleMelinda@mail.ru grotesqueRenee@hotmail.com Prestonprecious@t-online.de enchantingNatalie79@bellsouth.net shyDevin@zonnet.nl obedientKathleen@yahoo.es cleverLarry63@yahoo.com cruelRebecca62@hotmail.co.uk betterCody3@skynet.be zanyAna@hotmail.co.uk puzzledDavid@yahoo.com.ar Geoffreyfunny@yahoo.co.id zanyDenise19@arcor.de unusualMitchell21@cox.net drabThomas80@bigpond.net.au Meghanugliest@zonnet.nl comfortableGrace56@voila.fr upsetTristan@aol.com Mitchellpleasant@libero.it jitteryArthur24@live.com.au vivaciousBrittany@club-internet.fr Martinblack@live.co.uk jealousDominic@freenet.de easyShaun@mail.com toughSummer@facebook.com Meaganhomely@yahoo.com.au Marvinhappy@planet.nl longLeslie@verizon.net expensiveAdam@t-online.de Marctense@neuf.fr smoggyDevin57@live.ca Marcuscruel@verizon.net Catherinezany@bol.com.br lazyJacqueline85@sympatico.ca difficultAaron11@centurytel.net cuteMichael@t-online.de Nathanielclever@live.co.uk dizzyLee87@juno.com defeatedVirginia35@hotmail.es franticRonnie@yahoo.co.uk scaryAlan@yahoo.com.mx Kristinexcited@ntlworld.com dizzyBobby@hotmail.es inquisitiveAudrey61@frontiernet.net Ianlucky@sky.com Grantfrail@yahoo.co.in Jonathonhelpless@gmx.de ashamedTrevor92@facebook.com cuteAshlee@optusnet.com.au Kaitlinunsightly@yahoo.ca vivaciousJordan42@centurytel.net upsetMaria@gmx.de badLance@centurytel.net Jimmyfantastic@yahoo.co.id successfulCatherine74@virgilio.it cleverLisa@sympatico.ca Kristopherproud@blueyonder.co.uk magnificentGary@live.nl upsetChristie25@yahoo.fr blue-eyedVincent69@bluewin.ch angryJared@voila.fr Markhilarious@tiscali.co.uk repulsiveKathleen@neuf.fr nastyCandace40@t-online.de easyAaron@hotmail.it bluePeter48@verizon.net depressedLori78@yahoo.it Jerryblue-eyed@optonline.net toughHenry17@ntlworld.com disgustedLuis@sympatico.ca scaryErin@hetnet.nl Michealexuberant@planet.nl Virginiamushy@tin.it jealousCarolyn68@yahoo.co.in bewilderedJill53@laposte.net Troymotionless@virgilio.it perfectKristi57@telenet.be Margaretunusual@tiscali.it kindChristie@t-online.de sparklingPriscilla67@att.net"""
core_nlp(input_paragraph)


[EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL] [EMAIL]


In [60]:
# Testing pronouns:
input_paragraph = "he him his she her hers man woman men women"
core_nlp(input_paragraph)

 [THEY]  [THEM]  [THEIR]  [THEY]  [THEM]  [THEIR] [PERSON] [PERSON] [PEOPLE] [PEOPLE]


In [80]:
# Testing dates:
input_paragraph = "5/10/2022 5-10-2022 5-10-2022 5/10/2022 May 5, 2022 May 2022 - June 2022 1992-2023"
core_nlp(input_paragraph)

[DATE] [DATE] [DATE] [DATE] [DATE] [DATE] - [DATE] [DATE]


In [83]:
# Testing Company / Organization Names
input_paragraph = "I went to Mountain Vista High School. I went to Boulder High School. I went to University of Colorado Boulder. I went to Google. I worked at Facebook. I worked at Amazon."
core_nlp(input_paragraph)


I went to [ORGANIZATION]. I went to [ORGANIZATION]. I went to [ORGANIZATION]. I went to [ORGANIZATION]. I worked at [ORGANIZATION]. I worked at [ORGANIZATION].


In [84]:
extract_with_pdf_miner()
f = open("./sample_cover_letter.pdf.txt")
text = f.read()

In [85]:
print(text)

Sample Cover Letter

October 10, 2021

Susan Carey 
Senior Manager 
Wholesale Wine USA 
23 Green St. 
Boston, MA  02116 

Dear Susan Carey: 

I am writing to apply for your position in wine wholesale as advertised on Crimson Careers. This 
exciting opportunity appears to be a wonderful fit with my professional experience, personal 
interests, and career goals. 

I am returning to Boston to complete my final year at Harvard University Extension School, 
where I am majoring in French and economics. Having spent the year working and traveling, I am 
eager to incorporate myself once again into the local wine community, to which I can bring 
experience in a number of sectors of the industry.  

Through eight years in the restaurant field, I have acquired a deep love of and appreciation for 
wine and cuisine.  I have been known to wax rhapsodic over specials; nothing made me happier 
than discussing a bottle with a table. This enthusiasm allowed me to introduce a list of reserve 
selections 

In [86]:
# Testing on a resume
core_nlp(text)


Sample Cover Letter

[DATE]

[PERSON] 
Senior Manager 
Wholesale Wine [LOCATION] 
23 Green St. 
[LOCATION], [LOCATION]  02116 

Dear [PERSON]: 

I am writing to apply for your position in wine wholesale as advertised on [ORGANIZATION]. This 
exciting opportunity appears to be a wonderful fit with my professional experience, personal 
interests, and career goals. 

I am returning to [LOCATION] to complete my final year at [ORGANIZATION], 
where I am majoring in French and economics. Having spent the year working and traveling, I am 
eager to incorporate myself once again into the local wine community, to which I can bring 
experience in a number of sectors of the industry.  

Through eight years in the restaurant field, I have acquired a deep love of and appreciation for 
wine and cuisine.  I have been known to wax rhapsodic over specials; nothing made me happier 
than discussing a bottle with a table. This enthusiasm allowed me to introduce a list of reserve 
selections to [ORGANIZATIO

In [43]:
input_paragraph = "It was another early sunset on a rainy day in Seattle. Andrew was walking with a paper bag of groceries back to his downtown studio apartment at 400 Main Street. He had gotten government permission to stay in this apartment as part of the Upbeat program, focusing on bettering drug-affected neighborhoods in King County. It was an artist supplement program, and Andrew was a published haiku poet and teacher of haiku in public education and sometimes colleges. He had to attend an audition for artistic achievements to be able to get the subsidy. Though it was a miracle to live in downtown Seattle for $800 a month, it did have its downsides, with junkies visiting the weekly exhibitions of his haiku and accompanying minimalist art made by his neighbor, Patrick."
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
doc = nlp(input_paragraph)
# Doesn't include date, cardinal, maybe more
lst_entities = {
    'FAC': 'ADDRESS',
    'GPE': 'LOCATION',
    'PERSON': 'PERSON',
    'ORG': 'ORGANIZATION'
}
for sentence in doc.sentences:
#     print(sentence.ents)
    for i in sentence.ents:
        if i.type in lst_entities:
            input_paragraph = input_paragraph.replace(i.text,lst_entities[i.type])
replacements = {' he': ' THEY',
                      ' him': ' THEM',
                      ' his': ' THEIR',
                      ' she': ' THEY',
                      ' her': ' THEM',
                      ' hers': ' THEIR',
                     }

def replace(match):
    return replacements[match.group(0)]

# notice that the 'this' in 'thistle' is not matched 
regex = '|'.join(r'\b%s\b' % re.escape(s) for s in replacements)
print(regex)
new_txt = re.sub(regex, replace, input_paragraph)
print(new_txt)

# for key, value in lst_manual_changes.items():
#     print(key,value)
#     input_paragraph = input_paragraph.replace(key,value)
# print(input_paragraph)


2022-04-12 14:01:55 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-04-12 14:01:55 INFO: Use device: cpu
2022-04-12 14:01:55 INFO: Loading: tokenize
2022-04-12 14:01:55 INFO: Loading: pos
2022-04-12 14:01:55 INFO: Loading: lemma
2022-04-12 14:01:55 INFO: Loading: depparse
2022-04-12 14:01:55 INFO: Loading: sentiment
2022-04-12 14:01:56 INFO: Loading: constituency
2022-04-12 14:01:56 INFO: Loading: ner
2022-04-12 14:01:57 INFO: Done loading processors!


\b\ he\b|\b\ him\b|\b\ his\b|\b\ she\b|\b\ her\b|\b\ hers\b
It was another early sunset on a rainy day in LOCATION. PERSON was walking with a paper bag of groceries back to THEIR downtown studio apartment at ADDRESS. He had gotten government permission to stay in this apartment as part of the ORGANIZATION program, focusing on bettering drug-affected neighborhoods in LOCATION. It was an artist supplement program, and PERSON was a published haiku poet and teacher of haiku in public education and sometimes colleges. He had to attend an audition for artistic achievements to be able to get the subsidy. Though it was a miracle to live in downtown LOCATION for $800 a month, it did have its downsides, with junkies visiting the weekly exhibitions of THEIR haiku and accompanying minimalist art made by THEIR neighbor, PERSON.


In [44]:
names_info = pd.read_csv('us.txt')
names_info.columns = ['Names']
names_info = names_info[:500]
print(names_info)

      Names
0     Abbey
1     Abbie
2      Abby
3     Abdul
4       Abe
..      ...
495  Blaine
496   Blair
497   Blake
498  Blanca
499  Blanch

[500 rows x 1 columns]


In [73]:

def run_nlp(input_paragraph, looking_for=None):
    doc = nlp(input_paragraph)
    # Doesn't include date, cardinal, maybe more
    lst_entities = {
        'FAC': '[ADDRESS]',
        'GPE': '[LOCATION]',
        'PERSON': '[PERSON]',
        'ORG': '[ORGANIZATION]',
        'DATE': '[DATE]',
        'LOC': '[LOCATION]'
    }
    for sentence in doc.sentences:
        for i in sentence.ents:
            if i.type in lst_entities:
                if looking_for:
                    if i.type in looking_for:
                        return True
    return False
#                 input_paragraph = input_paragraph.replace(i.text,lst_entities[i.type])
    replacements = {' he': ' [THEY]',
                          ' him': ' [THEM]',
                          ' his': ' [THEIR]',
                          ' she': ' [THEY]',
                          ' her': ' [THEM]',
                          ' hers': ' [THEIR]',
                          'man': '[PERSON]',
                          'woman': '[PERSON]'
                         }

    def replace(match):
        return replacements[match.group(0)]

    # notice that the 'this' in 'thistle' is not matched 
    regex = '|'.join(r'\b%s\b' % re.escape(s) for s in replacements)
    new_txt = re.sub(regex, replace, input_paragraph)


    regex_email = '[a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+'
    lst_emails = re.findall(regex_email, new_txt)
    for email in lst_emails:
        new_txt = new_txt.replace(email,"[EMAIL]")

    regex_phone = '\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}'
    lst_phones = re.findall(regex_phone, new_txt)
    for phone in lst_phones:
#         print(phone)
        new_txt = new_txt.replace(phone,"[PHONE NUMBER]")

    regex_SSN = '[\d]{1,2}/[\d]{1,2}/[\d]{4}'
    lst_SSN = re.findall(regex_SSN, new_txt)
    for ssn in lst_SSN:
#         print(ssn)
        new_txt = new_txt.replace(ssn,"[DATE]")
#     print(new_txt)


    # 12-13-2020
    # SSN's
    # Citizenship Status

    # for key, value in lst_manual_changes.items():
    #     print(key,value)
    #     input_paragraph = input_paragraph.replace(key,value)
    # print(input_paragraph)


In [74]:
# Name data script:
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
cntr = 0
list_input = names_info['Names'][:1000]
lst_len = len(list_input)
for i in list_input:
    ret = run_nlp(i, ['PERSON'])
    if ret:
        cntr += 1
print(f"Percentage: {cntr/lst_len}")
# nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English


2022-04-12 17:52:36 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-04-12 17:52:36 INFO: Use device: cpu
2022-04-12 17:52:36 INFO: Loading: tokenize
2022-04-12 17:52:36 INFO: Loading: pos
2022-04-12 17:52:37 INFO: Loading: lemma
2022-04-12 17:52:37 INFO: Loading: depparse
2022-04-12 17:52:37 INFO: Loading: sentiment
2022-04-12 17:52:38 INFO: Loading: constituency
2022-04-12 17:52:38 INFO: Loading: ner
2022-04-12 17:52:39 INFO: Done loading processors!


Percentage: 0.912


In [69]:
# Script for Addresses
with open("addresses-us-100.json", "r") as read_file:
    data = json.load(read_file)
addresses = data['addresses']
lst_len = len(addresses)
cntr = 0
for address in addresses:
    print(address['address1'])
    
    ret = run_nlp(address['address1'], ['FAC','LOC','GPE'])
    if ret:
        cntr += 1
print(cntr,lst_len)
print(f"Percentage: {cntr/lst_len}")

1745 T Street Southeast
[]
6007 Applegate Lane
[{
  "text": "Applegate Lane",
  "type": "PERSON",
  "start_char": 5,
  "end_char": 19
}]
560 Penstock Drive
[{
  "text": "560",
  "type": "CARDINAL",
  "start_char": 0,
  "end_char": 3
}]
150 Carter Street
[{
  "text": "150",
  "type": "CARDINAL",
  "start_char": 0,
  "end_char": 3
}]
2721 Lindsay Avenue
[{
  "text": "Lindsay Avenue",
  "type": "PERSON",
  "start_char": 5,
  "end_char": 19
}]
18 Densmore Drive
[{
  "text": "18",
  "type": "CARDINAL",
  "start_char": 0,
  "end_char": 2
}]
637 Britannia Drive
[{
  "text": "637 Britannia Drive",
  "type": "FAC",
  "start_char": 0,
  "end_char": 19
}]
5601 West Crocus Drive
[{
  "text": "5601 West Crocus Drive",
  "type": "FAC",
  "start_char": 0,
  "end_char": 22
}]
5403 Illinois Avenue
[{
  "text": "Illinois Avenue",
  "type": "FAC",
  "start_char": 5,
  "end_char": 20
}]
8821 West Myrtle Avenue
[{
  "text": "8821 West Myrtle Avenue",
  "type": "FAC",
  "start_char": 0,
  "end_char": 23
}]


[{
  "text": "6231 North 67th Avenue",
  "type": "FAC",
  "start_char": 0,
  "end_char": 22
}]
8505 Waters Avenue
[]
7 Underwood Place Northwest
[{
  "text": "7",
  "type": "CARDINAL",
  "start_char": 0,
  "end_char": 1
}]
21950 Arnold Center Road
[{
  "text": "Arnold Center Road",
  "type": "FAC",
  "start_char": 6,
  "end_char": 24
}]
1427 South Carolina Avenue Southeast
[{
  "text": "South Carolina",
  "type": "GPE",
  "start_char": 5,
  "end_char": 19
}, {
  "text": "Avenue Southeast",
  "type": "LOC",
  "start_char": 20,
  "end_char": 36
}]
1420 Turtleback Trail
[]
29 99
Percentage: 0.29292929292929293


In [48]:
input_paragraph = "(303)888 8888 303-884-6663 Boulder High School May 26th bryce.schumacher8@gmail.com 11/15/2020 University of Colorado 11/13/2020 man woman His nose could have been mistaken for a carrot. Not one of those dirt-rich oblong carrots, but a baby-sized carrot packed in those free-moving, punctured plastic bags. Yet more distinguishable were his cheeks, slit in as if a wild animal unintentionally scathed a straight line down the skin by his cheekbone. A rugged Pinocchio if I ever saw one. But his name wasn’t that beefy: Marty. Marty was a name white collar folk call in for Sunday dinner for tidbits of entertainment or a chiseled smile. Jared seemed like a more convincing name for him, especially since that nose of his could have poked Napoleon back to eating crêpes—but they would have it that this gritty-faced man (we haven’t gotten into his personality yet) was named Marty at birth. No, it’s not what you think—it isn’t Martin. Parents do the darndest things."
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
doc = nlp(input_paragraph)
# Doesn't include date, cardinal, maybe more
lst_entities = {
    'FAC': '[ADDRESS]',
    'GPE': '[LOCATION]',
    'PERSON': '[PERSON]',
    'ORG': '[ORGANIZATION]',
    'DATE': '[DATE]'
}
for sentence in doc.sentences:
#     print(sentence.ents)
    for i in sentence.ents:
        if i.type in lst_entities:
            input_paragraph = input_paragraph.replace(i.text,lst_entities[i.type])
replacements = {' he': ' [THEY]',
                      ' him': ' [THEM]',
                      ' his': ' [THEIR]',
                      ' she': ' [THEY]',
                      ' her': ' [THEM]',
                      ' hers': ' [THEIR]',
                      'man': '[PERSON]',
                      'woman': '[PERSON]'
                     }

def replace(match):
    return replacements[match.group(0)]

# notice that the 'this' in 'thistle' is not matched 
regex = '|'.join(r'\b%s\b' % re.escape(s) for s in replacements)
new_txt = re.sub(regex, replace, input_paragraph)


regex_email = '[a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+'
lst_emails = re.findall(regex_email, new_txt)
for email in lst_emails:
    new_txt = new_txt.replace(email,"[EMAIL]")
    
regex_phone = '\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}'
lst_phones = re.findall(regex_phone, new_txt)
for phone in lst_phones:
    print(phone)
    new_txt = new_txt.replace(phone,"[PHONE NUMBER]")

regex_SSN = '[\d]{1,2}/[\d]{1,2}/[\d]{4}'
lst_SSN = re.findall(regex_SSN, new_txt)
for ssn in lst_SSN:
    print(ssn)
    new_txt = new_txt.replace(ssn,"[DATE]")
print(new_txt)


# 12-13-2020
# SSN's
# Citizenship Status

# for key, value in lst_manual_changes.items():
#     print(key,value)
#     input_paragraph = input_paragraph.replace(key,value)
# print(input_paragraph)


2022-04-12 14:02:19 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-04-12 14:02:19 INFO: Use device: cpu
2022-04-12 14:02:19 INFO: Loading: tokenize
2022-04-12 14:02:19 INFO: Loading: pos
2022-04-12 14:02:20 INFO: Loading: lemma
2022-04-12 14:02:20 INFO: Loading: depparse
2022-04-12 14:02:21 INFO: Loading: sentiment
2022-04-12 14:02:21 INFO: Loading: constituency
2022-04-12 14:02:22 INFO: Loading: ner
2022-04-12 14:02:22 INFO: Done loading processors!


(303)888 8888
303-884-6663
11/15/2020
11/13/2020
[PHONE NUMBER] [PHONE NUMBER] [ORGANIZATION] [DATE] [EMAIL] [DATE] [ORGANIZATION] [DATE] [PERSON] [PERSON] His nose could have been mistaken for a carrot. Not one of those dirt-rich oblong carrots, but a baby-sized carrot packed in those free-moving, punctured plastic bags. Yet more distinguishable were [THEIR] cheeks, slit in as if a wild animal unintentionally scathed a straight line down the skin by [THEIR] cheekbone. A rugged [PERSON] if I ever saw one. But [THEIR] name wasn’t that beefy: [PERSON]. [PERSON] was a name white collar folk call in for [DATE] dinner for tidbits of entertainment or a chiseled smile. [PERSON] seemed like a more convincing name for [THEM], especially since that nose of [THEIR] could have poked [PERSON] back to eating crêpes—but they would have it that this gritty-faced [PERSON] (we haven’t gotten into [THEIR] personality yet) was named [PERSON] at birth. No, it’s not what you think—it isn’t [PERSON]. Parents